# UTS Analisis Teks
# Eko Ikhwan Saputra

Link github : https://github.com/ulung3ko/uts-analisis-teks

In [1]:
# Import Library yang diubutuhkan
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Bagian 0
# Artikel Terkini
Artikel yang diangkat berjudul "Pemain Timnas Indonesia Terbang ke Bali, Bersiap Bela Garuda" dari CNN Indonesia. Artikel berisi kabar terbaru tentang persiapan Timnas Indonesia untuk Kualifikasi Piala Dunia 2026.  
Sumber : https://www.cnnindonesia.com/olahraga/20250525071942-142-1232869/pemain-timnas-indonesia-terbang-ke-bali-bersiap-bela-garuda

# Isu Permasalahan
Topik Timnas Indonesia secara konsisten menjadi perhatian publik di media sosial terutama menjelang pertandingan. Di balik semangat dukungan, muncul juga kritik terhadap performa pemain, strategi pelatih, dan keputusan federasi. Cara konvensional susah untuk memahami kecenderungan sentimen publik.

# Solusi dengan Klasifikasi Teks
Permasalahan ini dapat diselesaikan dengan membangun model klasifikasi teks yang secara otomatis mengidentifikasi sentimen publik (positif atau negatif) terhadap Timnas Indonesia. Model akan membantu dalam menganalisis opini masyarakat secara cepat dan sistematis sebagai bahan evaluasi dan strategi komunikasi publik.


# Bagian 1
Dataset yang digunakan adalah "Analisis Sentimen Timnas Sepakbola di Era STY" dari Kaggle, terdiri dari.
- 398 baris data.
- 2 kolom: komentar dan label.
- Label berupa positive dan negative.

Memang judul dataset menyebut era STY,  tapi dataset ini tetap relevan karena berisi opini publik terhadap Timnas secara umum.

Sumber : https://www.kaggle.com/datasets/pajarbebek/analisis-sentimen-timnas-sepakbola-di-era-sty

In [2]:
# Baca dan lihat 5 baris dataser
df = pd.read_csv('data_sentiment_timnas.csv')
df.head()

,komentar,label
0,<username> Simulasi indonesia kembali dijajah ...,negatif
1,<username> <username> <username> keturunan lai...,negatif
2,<username> <username> uda botak keturunan penj...,negatif
3,"<username> <username> <username> Belum tentu, ...",negatif
4,<username> <username> <username> Lu nya aja yg...,negatif


In [3]:
# Melihat statistik dasar dataset
jumlah_kategori = df['label'].nunique()
print(f"Jumlah kategori unik dalam kolom 'label': {jumlah_kategori}")

kategori_unik = df['label'].unique()
print(f"Kategori unik dalam kolom 'label': {kategori_unik}")

print("\nInformasi Dataset:")
df.info()

print("\nDeskripsi Statistik Dataset:")
print(df.describe(include='all'))

Jumlah kategori unik dalam kolom 'label': 2
Kategori unik dalam kolom 'label': ['negatif' 'positif']

Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   komentar  398 non-null    object
 1   label     398 non-null    object
dtypes: object(2)
memory usage: 6.3+ KB

Deskripsi Statistik Dataset:
                                                 komentar    label
count                                                 398      398
unique                                                393        2
top     <username> Dendy sama arhan punya aib nya sty ...  positif
freq                                                    2      204


# Bagian 2
Pra-premsosesan Teks dilakukan sesuai dengan instruksi yaitu.
- Tokenisasi.
- Penghapusan stopwords.
- Stemming.
- Case Folding.
- Hapus tanda baca dan karakter khusus.

In [4]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
list_stopwords_nltk = stopwords.words('indonesian')
custom_stopwords = ['username', 'nya', 'ma', 'tp', 'mah', 'jg', 'sih', 'tdk', 'utk', 'deh', 'jd', 'lg', 'gk', 'ga', 'gak', 'yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 'krn'] 
list_stopwords = set(list_stopwords_nltk + custom_stopwords)

# Fungsi Pre-processing
def preprocess_text(text):
    # Memastikan input adalah string dan menangani nilai NaN
    if pd.isna(text):
        return ""
    text = str(text)

    # 1. Lowercasing: Mengubah semua teks menjadi huruf kecil
    text = text.lower()

    # 2. Hapus tanda baca dan karakter khusus (mempertahankan huruf dan spasi)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.strip()

    # 3. Tokenisasi: Memecah teks menjadi token (kata)
    tokens = word_tokenize(text)

    # 4. Penghapusan Stopwords
    tokens_without_stopwords = [word for word in tokens if word not in list_stopwords and len(word) > 1] # Opsi: len(word)>1 untuk hapus token 1 huruf

    # 5. Stemming: Mengubah kata menjadi bentuk dasarnya
    if 'stemmer' in globals():
        stemmed_tokens = [stemmer.stem(word) for word in tokens_without_stopwords]
    else:
        stemmed_tokens = tokens_without_stopwords # Lanjutkan tanpa stemming jika Sastrawi gagal

    # Menggabungkan kembali token menjadi satu string teks
    processed_text = " ".join(stemmed_tokens)
    return processed_text

print("DataFrame Asli (beberapa baris pertama):")
print(df.head())
print("-" * 50)

# 2. Apply  ke kolom 'komentar'
print("\nMemulai pra-pemrosesan pada kolom 'komentar'...")
df['komentar_bersih'] = df['komentar'].apply(preprocess_text)
print("Pra-pemrosesan selesai.")
print("-" * 50)

DataFrame Asli (beberapa baris pertama):
                                            komentar    label
0  <username> Simulasi indonesia kembali dijajah ...  negatif
1  <username> <username> <username> keturunan lai...  negatif
2  <username> <username> uda botak keturunan penj...  negatif
3  <username> <username> <username> Belum tentu, ...  negatif
4  <username> <username> <username> Lu nya aja yg...  negatif
--------------------------------------------------

Memulai pra-pemrosesan pada kolom 'komentar'...
Pra-pemrosesan selesai.
--------------------------------------------------


In [6]:
# Hasil pre-processing
print(df.head())
print("-" * 50)

                                            komentar    label  \
0  <username> Simulasi indonesia kembali dijajah ...  negatif   
1  <username> <username> <username> keturunan lai...  negatif   
2  <username> <username> uda botak keturunan penj...  negatif   
3  <username> <username> <username> Belum tentu, ...  negatif   
4  <username> <username> <username> Lu nya aja yg...  negatif   

                                     komentar_bersih  
0  simulasi indonesia jajah turun belanda depan b...  
1  turun main bola emang naturalisasi istimewa pe...  
2  uda botak turun jajah sok si paham bola lu coa...  
3  main bola milih karir main bola kualitas timna...  
4  lu aja bego nonton tolol persepsi orang beda n...  
--------------------------------------------------
